# Enunciado
Hola ingeniero 👋. Bienvenido a tu sexto desafio! Ya sabes que en [Vault-Tec Corporation](https://fallout.fandom.com/es/wiki/Vault-Tec_Corporation) tenemos sensores encargados de monitorear las condiciones actuales de nuestros refugios. En ocasiones pasadas detectamos que varios de ellos presentaban fallas recurrentes, impidiendo que pudieramos determinar las condiciones de nuestros bóvedas y darle la seguridad esperada a nuestros clientes en un mundo post apocaliptico 🙃. 

Anteriormente, a pedido de nuestro equipo, solucionamos algunos errores detectados en el funcionamiento de nuestro codigo y generamos pruebas unitarias; ademas, modificamos el codigo para que usara vectores de Numpy en lugar de listas nativas de Python, evidenciando una clara mejora en los tiempos de ejecucion tomados. 

Uno de los desarrolladores Senior nos ha sugerido hacer pruebas de rendimiento usando el modulo Pandas. Para ello nos ha indicado que modifiquemos todas las partes del codigo que usan vectores de numpy por **Dataframes de Pandas.**

# Insumos
Tu equipo te ha dado el siguiente codigo el cual deberas modificar. 

**IMPORTANTE:** En la vida real lo comun es que pases mas tiempo leyendo codigo de otros, que escribiendolo. 

In [1]:
import json
from datetime import date
from os.path import abspath

import numpy as np


class NumpyLogExtractor:
    """Clase para extraer y filtrar logs desde un archivo JSON."""

    def __init__(self, file_path: str):
        """
        Inicializa el extractor de logs con la ruta del archivo.

        Args:
            file_path (str): Ruta del archivo JSON que contiene los logs.
        """
        self.file_abspath = abspath(file_path)

    def from_file(self) -> np.ndarray:
        """
        Carga los logs desde el archivo JSON especificado en la inicialización.

        Returns:
            list[Log]: Lista de instancias de Log extraídas del archivo.
        """
        log_dtype: list[tuple] = [
            ("sensor_id", "U16"),
            ("date", "M8[D]"),  # fecha con resolución de días
            ("event_type", "U16"),
            ("duration_seconds", "f8"),
        ]

        with open(self.file_abspath) as file:
            raw_logs: list[dict] = json.load(file)
        logs = np.array(
            [
                (
                    log["sensor_id"],
                    np.datetime64(log["timestamp"]),
                    log["event_type"],
                    float(log["duration_seconds"]),
                )
                for log in raw_logs
            ],
            dtype=log_dtype,
        )
        return logs

    @staticmethod
    def fetch_logs_on_date(
        target_date: date, logs: np.ndarray, sensor_id: int | None = None
    ) -> np.ndarray:
        """
        Filtra los logs que corresponden a una fecha y, opcionalmente, a un sensor específico.

        Args:
            target_date (date): Fecha objetivo para filtrar los logs.
            logs (list[Log]): Lista de logs disponibles.
            sensor_id (Optional[int], optional): ID del sensor para filtrar.
                Si es None, se incluyen logs de todos los sensores. Defaults to None.

        Returns:
            list[Log]: Lista de logs que cumplen con los filtros aplicados.
        """
        mask = logs["date"] == np.datetime64(target_date)
        if sensor_id is not None:
            mask &= logs["sensor_id"] == sensor_id
        return logs[mask]

In [2]:
from datetime import date, timedelta

import numpy as np


class NumpySensorFailureAnalyzer:
    """Clase para analizar fallos de sensores a partir de registros de eventos usando NumPy."""

    WEEK_TOTAL_SECONDS: int = 7 * 24 * 60 * 60
    """Número total de segundos en una semana."""

    def __init__(self, logs: np.ndarray):
        """Inicializa el analizador con los registros de logs.

        Args:
            logs (np.ndarray): Array estructurado de logs con dtype:
                [
                    ("sensor_id", "i4"),
                    ("date", "M8[D]"),
                    ("event_type", "U16"),
                    ("duration_seconds", "f8"),
                ]
        """
        self.logs = logs

    def get_sensor_failure_duration(self, sensor_id: int, searched_date: date) -> float | None:
        """Obtiene la duración total del fallo de un sensor en una fecha específica."""
        mask = (self.logs["sensor_id"] == sensor_id) & (
            self.logs["date"].astype("datetime64[D]") == np.datetime64(searched_date)
        )
        logs_on_date = self.logs[mask]

        if logs_on_date.size == 0:
            return None

        failure_start = logs_on_date[logs_on_date["event_type"] == "FAILURE_START"][
            "duration_seconds"
        ]
        failure_end = logs_on_date[logs_on_date["event_type"] == "FAILURE_END"]["duration_seconds"]

        start_time = failure_start[0] if failure_start.size > 0 else 0.0
        end_time = failure_end[0] if failure_end.size > 0 else 0.0

        return abs(end_time - start_time)

    def get_weekly_sensor_failure_duration(
        self, sensor_id: int, start_date: date, end_date: date | None = None
    ) -> list[dict]:
        """Obtiene la duración diaria de fallos de un sensor durante una semana."""
        end_date = start_date + timedelta(7) if end_date is None else end_date
        searched_week = [
            start_date + timedelta(days=i) for i in range((end_date - start_date).days)
        ]

        return [
            {
                "sensor_id": sensor_id,
                "date": str(day),
                "failure_seconds": self.get_sensor_failure_duration(sensor_id, day),
            }
            for day in searched_week
        ]

    def get_weekly_sensor_failure_probability(self, sensor_id: int, start_date: date) -> list[dict]:
        """Calcula la probabilidad diaria de fallo de un sensor durante una semana."""
        weekly_durations = self.get_weekly_sensor_failure_duration(sensor_id, start_date)
        return [
            {
                "sensor_id": sensor_id,
                "date": x["date"],
                "failure_probability": f"{(x['failure_seconds'] or 0) / self.WEEK_TOTAL_SECONDS:.4f}",
            }
            for x in weekly_durations
        ]

    def get_conditional_failure_probability(
        self,
        sensor_a_id: int,
        sensor_b_id: int,
        start_date: date,
        failure_minutes_threshold: float | None = 3,
    ) -> float:
        """Calcula la probabilidad condicional de que un sensor falle dado que otro sensor falló."""
        threshold = failure_minutes_threshold * 60

        sensor_b_week = self.get_weekly_sensor_failure_duration(sensor_b_id, start_date)
        count_b_failures = sum(
            1 for x in sensor_b_week if x["failure_seconds"] and x["failure_seconds"] >= threshold
        )
        if count_b_failures == 0:
            return 0.0

        sensor_a_week = self.get_weekly_sensor_failure_duration(sensor_a_id, start_date)
        count_a_b_failures = sum(
            1
            for x, y in zip(sensor_a_week, sensor_b_week, strict=False)
            if x["failure_seconds"]
            and y["failure_seconds"]
            and x["failure_seconds"] >= threshold
            and y["failure_seconds"] >= threshold
        )

        return count_a_b_failures / count_b_failures

# Tu turno!

Ahora es tu turno! Evaluemos el rendimiento de nuestro codigo actual contra la modificacion sugerida por nuestro compañero de equipo.

## 1. Benchmarking de LogExtractor
Reescribamos, probemos y comparemos los tiempos de ejecucion de la clase `NumpyLogExtractor` que tenemos en este momento, contra una version de esta clase modificada para usar Dataframes de Pandas.

**Reescribiendo la clase NumpyLogExtractor:** Reescribe la clase NumpyLogExtractor para que use Dataframes de Pandas en lugar de vectores de Numpy.
1. Nombra la nueva clase PandasLogExtractor. 
2. Modifica las funciones de esta clase que retornan `numpy.ndarray` para que retornen `pandas.DataFrame`.

In [ ]:
# Tu codigo aqui

Comparemos el tiempo de ejecucion de nuestra clase NumpyLogExtractor pasada contra esta nueva implementacion en Pandas para la funcion `fetch_logs_on_date`.
- Consejo 1: Extrae los logs con ambas clases en una celda diferente a donde vayas a ejecutar `fetch_logs_on_date`; de no hacerlo, la medida del tiempo de ejecucion se vera contaminada con el tiempo tomado por la funcion de extraccion de logs.
- Consejo 2: Busca sobre magics proporcionadas por Jupyter para medicion de tiempos y como usarlas. Consulta sobre `%%timeit`.

In [ ]:
# Tu codigo aqui

## 2. Benchmarking de SensorFailureAnalyzer

Reescribamos, probemos y comparemos los tiempos de ejecucion de la clase `NumpySensorFailureAnalyzer` que tenemos en este momento, contra una version de esta clase modificada para usar dataframes de Pandas.

**Reescribiendo la clase SensorFailureAnalyzer:** Reescribe la clase NumpySensorFailureAnalyzer para que use Datafarmes de Pandas en lugar de vectores de Numpy
1. Nombra la nueva clase PandasSensorFailureAnalyzer. 

In [ ]:
# Tu codigo aqui

Comparemos el tiempo de ejecucion de nuestra clase SensorFailureAnalyzer pasada contra esta nueva implementacion en Numpy para la funcion `get_weekly_sensor_failure_probability`.
- Consejo 1: Usa los logs anteriormente extraidos e instancia (crea) ambas clases en una celda diferente a donde vayas a ejecutar `get_weekly_sensor_failure_probability`; de no hacerlo, la medida del tiempo de ejecucion se vera contaminada con el tiempo tomado para crear las clases.

In [ ]:
# Tu codigo aqui